In [8]:
# Question 1

from itertools import combinations
from functools import partial
from pprint import pprint as pp

def generate_input(attrs_by_obj):
    objs_by_attr = dict()
    for obj, attrs in attrs_by_obj.iteritems():
        for attr in attrs:
            if attr not in objs_by_attr:
                objs_by_attr[attr] = set()
            objs_by_attr[attr].add(obj)
    
    attributes = reduce(lambda a, b: a.union(b), attrs_by_obj.values())
    objects = sorted(attrs_by_obj.keys())
    
    attributes_set = set(attributes)
    objects_set = set(objects)
    
    return attributes, attributes_set, objs_by_attr, objects, objects_set, attrs_by_obj
    

def obj_closure(objects, all_attributes, attrs_by_obj):
    m = set(all_attributes)
    for obj in objects:
        m = m.intersection(attrs_by_obj[obj])
    return m

def attr_closure(attributes, all_objects, objs_by_attr):
    g = set(all_objects)
    for attr in attributes:
        g = g.intersection(objs_by_attr[attr])
    return g


def closure(attributes, all_attributes, all_objects, attrs_by_obj, objs_by_attr):
    return obj_closure(attr_closure(attributes, all_objects, objs_by_attr), all_attributes, attrs_by_obj)


def get_pseudo_intents(attributes_set, get_closure):
    pseudo_intents = list()
    
    for i in xrange(0, len(attributes_set) + 1):
        for subset in list(combinations(attributes_set, i)):
            subset = set(subset)            
            if subset != get_closure(subset):
                is_pseudo_intent = True
                for pseudo_intent in pseudo_intents:
                    if pseudo_intent < subset:
                        if not get_closure(pseudo_intent) <= subset:
                            is_pseudo_intent = False
                            break
                if is_pseudo_intent:
                    pseudo_intents.append(subset)
    return pseudo_intents

def get_preclosure(intent_set, pseudo_intents, get_closure):
    prev_intents = None
    cur_intents = intent_set
    while prev_intents != cur_intents:
        pseudo_intents_union = set()
        for pseudo_intent in pseudo_intents:
            if pseudo_intent < cur_intents:
                pseudo_intents_union = pseudo_intents_union.union(get_closure(pseudo_intent))
        prev_intents = cur_intents
        cur_intents = cur_intents.union(pseudo_intents_union)
    return cur_intents

def next_closure(attribute_subset, attributes, get_closure):
    s = set(attribute_subset)
    for a in reversed(attributes):
        if a in s:
            s = s.difference(set([a]))
        else:
            b = get_closure(s.union(set([a])))
            diff = b.difference(s)
            contains_smaller = any(attributes.index(i) < attributes.index(a) for i in diff)
            if not contains_smaller:
                return b
    return set(attributes)

def get_canonical_basis(attributes, get_closure, get_preclosure):
    el = []
    a = set()
    attributes_set = set(attributes)
    while a != attributes_set:
        a_closure = get_closure(a) 
        if a != a_closure:
            el.append([a, '->', a_closure.difference(a)])
        a = next_closure(a, attributes, get_preclosure)
    return el

    
def calculate_answer(attrs_by_obj):
    all_attributes, attributes_set, objs_by_attr, all_objects, objects_set, attrs_by_obj = generate_input(attrs_by_obj)
    
    prefilled_closure = lambda attributes: closure(attributes, all_attributes, all_objects, attrs_by_obj, objs_by_attr) 
    pseudo_intents = get_pseudo_intents(attributes_set, prefilled_closure)
    preclosure = lambda intent_set: get_preclosure(intent_set, pseudo_intents, prefilled_closure)
    basis = get_canonical_basis(sorted(all_attributes), prefilled_closure, preclosure)
    pp(len(basis))
    pp(basis)


In [2]:
# Question 1

def base_context():
    attrs_by_obj = {
        'T1': set(['Na Rich',   'Ca Medium', 'Al Medium', 'Fe Medium', 'Cu Medium']),
        'T2': set(['Na Medium', 'Ca Rich',   'Al Rich',   'Fe Medium', 'Cu Weak']),
        'T3': set(['Na Rich',   'Ca Rich',   'Al Rich',   'Fe Medium', 'Cu Weak']),
        'T4': set(['Na Rich',   'Ca Medium', 'Al Medium', 'Fe Weak',   'Cu Rich']),
        'T5': set(['Na Rich',   'Ca Rich',   'Al Rich',   'Fe Medium', 'Cu Weak']),
        'T6': set(['Na Rich',   'Ca Medium', 'Al Medium', 'Fe Medium', 'Cu Very Weak']),
    }
    pairs = list(combinations(attrs_by_obj.keys(), 2))
    new_attrs_by_obj = dict()
    for pair in pairs:
        a, b = pair
        new_attrs_by_obj[pair] = attrs_by_obj[a] & attrs_by_obj[b]
    
    short_attrs_by_obj = dict()
    for pair, attributes in new_attrs_by_obj.iteritems():
        attributes = list(attributes)
        for i in xrange(0, len(attributes)):
            attributes[i] = attributes[i].split(' ')[0]
        short_attrs_by_obj[pair] = set(attributes)
    
    all_attributes, attributes_set, objs_by_attr, all_objects, objects_set, attrs_by_obj = generate_input(short_attrs_by_obj)
    calculate_answer(attrs_by_obj)

base_context() 

# We expect canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]


3
[[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
 [set(['Ca']), '->', set(['Al'])],
 [set(['Al']), '->', set(['Ca'])]]


In [3]:
# Question 1

def context1():
    attrs_by_obj = {
        '1': set(['Fe']),
        '2': set(['Na', 'Fe']),
        '3': set(['Na', 'Ca']),
        '4': set(['Na', 'Ca', 'Al', 'Fe']),
        '5': set(['Ca', 'Al', 'Fe', 'Cu']),
        '6': set([]),
        '7': set(['Na']),
        '8': set(['Na', 'Ca', 'Al', 'Fe', 'Cu']),
        '9': set(['Na', 'Cu']),
    }
    calculate_answer(attrs_by_obj)

context1()
# Got canonical basis:
# [[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
#  [set(['Ca', 'Fe']), '->', set(['Al'])],
#  [set(['Ca', 'Cu']), '->', set(['Al', 'Fe'])],
#  [set(['Al']), '->', set(['Ca', 'Fe'])]]

# We expect canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]

4
[[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
 [set(['Ca', 'Fe']), '->', set(['Al'])],
 [set(['Ca', 'Cu']), '->', set(['Al', 'Fe'])],
 [set(['Al']), '->', set(['Ca', 'Fe'])]]


In [4]:
# Question 1

def context2():
    attrs_by_obj = {
        '1': set(['Al', 'Fe']),
        '2': set(['Na', 'Fe']),
        '3': set(['Na', 'Ca']),
        '4': set(['Na', 'Ca', 'Al', 'Fe']),
        '5': set(['Ca', 'Al', 'Fe', 'Cu']),
        '6': set([]),
        '7': set(['Na']),
        '8': set(['Na', 'Ca', 'Al', 'Fe', 'Cu']),
        '9': set(['Na', 'Cu']),
    }
    calculate_answer(attrs_by_obj)

context2()
# Got canonical basis:
# [[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
#  [set(['Ca', 'Fe']), '->', set(['Al'])],
#  [set(['Ca', 'Cu']), '->', set(['Al', 'Fe'])],
#  [set(['Al']), '->', set(['Fe'])],
#  [set(['Al', 'Fe', 'Na']), '->', set(['Ca'])]]

# We expect canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]

5
[[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
 [set(['Ca', 'Fe']), '->', set(['Al'])],
 [set(['Ca', 'Cu']), '->', set(['Al', 'Fe'])],
 [set(['Al']), '->', set(['Fe'])],
 [set(['Al', 'Fe', 'Na']), '->', set(['Ca'])]]


In [5]:
# Question 1

def context3():
    attrs_by_obj = {
        '1': set(['Fe']),
        '2': set(['Na', 'Fe']),
        '3': set(['Na', 'Ca', 'Al']),
        '4': set(['Na', 'Ca', 'Al', 'Fe']),
        '5': set(['Ca', 'Al', 'Fe', 'Cu']),
        '6': set(['Al', 'Cu']),
        '7': set(['Na']),
        '8': set(['Na', 'Ca', 'Al', 'Fe', 'Cu']),
        '9': set(['Na', 'Cu']),
    }
    calculate_answer(attrs_by_obj)

context3()
# Got canonical basis:
# [[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al', 'Na']), '->', set(['Ca'])],
#  [set(['Al', 'Fe']), '->', set(['Ca'])],
#  [set(['Al', 'Ca', 'Cu']), '->', set(['Fe'])]]

# We expect canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]

5
[[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
 [set(['Ca']), '->', set(['Al'])],
 [set(['Al', 'Na']), '->', set(['Ca'])],
 [set(['Al', 'Fe']), '->', set(['Ca'])],
 [set(['Al', 'Ca', 'Cu']), '->', set(['Fe'])]]


In [6]:
# Question 1

def context4():
    attrs_by_obj = {
        '1': set(['Fe']),
        '2': set(['Na', 'Fe']),
        '3': set(['Na', 'Ca', 'Al']),
        '4': set(['Na', 'Ca', 'Al', 'Fe']),
        '5': set(['Ca', 'Al', 'Fe', 'Cu']),
        '6': set([]),
        '7': set(['Na']),
        '8': set(['Na', 'Ca', 'Al', 'Fe', 'Cu']),
        '9': set(['Na', 'Fe']),
    }
    calculate_answer(attrs_by_obj)

context4()    
# Got canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]

# We expect canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]

3
[[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
 [set(['Ca']), '->', set(['Al'])],
 [set(['Al']), '->', set(['Ca'])]]


In [7]:
# Question 1
def context5():
    attrs_by_obj = {
        '1': set(['Fe']),
        '2': set(['Na', 'Fe']),
        '3': set(['Na', 'Ca', 'Al']),
        '4': set(['Na', 'Ca', 'Al', 'Fe']),
        '5': set(['Ca', 'Al', 'Fe', 'Cu']),
        '6': set([]),
        '7': set(['Na']),
        '8': set(['Na', 'Ca', 'Al', 'Fe', 'Cu']),
        '9': set(['Na', 'Cu']),
    }
    calculate_answer(attrs_by_obj)

context5()

# Got canonical basis:
# [[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])],
#  [set(['Al', 'Ca', 'Cu']), '->', set(['Fe'])]]

# We expect canonical basis:
# [[set(['Cu']), '->', set(['Al', 'Ca', 'Fe'])],
#  [set(['Ca']), '->', set(['Al'])],
#  [set(['Al']), '->', set(['Ca'])]]

4
[[set(['Cu', 'Fe']), '->', set(['Al', 'Ca'])],
 [set(['Ca']), '->', set(['Al'])],
 [set(['Al']), '->', set(['Ca'])],
 [set(['Al', 'Ca', 'Cu']), '->', set(['Fe'])]]
